In [20]:
import pickle
import pandas as pd
import json 
import numpy as np
from weather_api import DarkSkyApi, SolCastApi

MODEL_DIR = '../../models'

with open('../../data/production_data/site_metadata.json', 'r') as file:
    s = file.read()

site_md = json.loads(s)



In [17]:
from calendar import isleap

def get_site_by_id(site_id):
    for site in site_md:
        if site['id'] == site_id:
            return site 

def load_prod_model(site_id):
    filename = f'{MODEL_DIR}/production/{site_id}/v1.sav'
    model = pickle.load(open(filename, 'rb'))
    return model 

def get_weather_forecast_df(lat, lon):
    ds_api = DarkSkyApi(True)
    forecast = ds_api.get_forecast(lat, lon)
    data = forecast['hourly']['data']
    df = pd.DataFrame(data)
    # df['time'] = df.apply(lambda row: row['time'] * 1000, axis=1)
    df['time'] = pd.to_datetime(df['time'], unit='s', utc=True)
    df = df.set_index('time')
    return df 

def get_irr_forecast_df(lat, lon):
    sc_api = SolCastApi(True)
    forecast = sc_api.get_forecast(lat, lon)
    data = forecast['forecasts']
    df = pd.DataFrame(data)
    df['time'] = pd.to_datetime(df['period_end'])
    df = df.set_index('time')
    df = df.resample('1H').mean()
    return df 

def day_trans(row):
    year = row.name.year 
    day_of_year = row.name.dayofyear
    days_in_year = 366 if isleap(year) else 365
    return np.sin(day_of_year * np.pi / days_in_year)

def add_seasonality(df):
    df['day'] = df.apply(day_trans, axis=1)
    df['hour'] = df.apply(lambda row: row.name.hour * np.pi / 24, axis=1)
    return df 

In [18]:
site = get_site_by_id(103941)
lat = site['location']['lat']
lon = site['location']['long']
weather_forecast_df = get_weather_forecast_df(lat, lon)
irr_forecast_df = get_irr_forecast_df(lat, lon)

Making request to url: https://api.darksky.net/forecast/09af10825ba1cd8092ba13b76c911acd/36.9587533,-112.9758274
Making request to url: https://api.solcast.com.au/world_radiation/forecasts?latitude=36.9587533&longitude=-112.9758274&api_key=DH7r5R0ctJhIyHxnprkKEwjGcQl6cuR9&format=json


In [21]:
join_df = weather_forecast_df.merge(irr_forecast_df, how='inner', left_index=True, right_index=True)
# display(weather_forecast_df)
# display(irr_forecast_df)
join_df = add_seasonality(join_df)
join_df

,summary,icon,precipIntensity,precipProbability,temperature,apparentTemperature,dewPoint,humidity,pressure,windSpeed,...,dni,dni10,dni90,dhi,air_temp,zenith,azimuth,cloud_opacity,day,hour
time,,,,,,,,,,,,,,,,,,,,,
2020-05-04 21:00:00+00:00,Clear,clear-day,0,0,76.73,76.73,13.06,0.09,1014.6,9.36,...,974.5,940.0,974.5,93.5,22.0,29.5,130.5,0.0,0.878612,2.748894
2020-05-04 22:00:00+00:00,Clear,clear-day,0,0,78.13,78.13,8.72,0.07,1013.7,9.58,...,946.5,920.0,946.5,84.0,22.0,39.5,112.0,0.0,0.878612,2.879793
2020-05-04 23:00:00+00:00,Clear,clear-day,0,0,79.52,79.52,4.21,0.05,1013.8,9.71,...,882.5,824.0,896.0,89.5,21.0,51.0,99.5,0.5,0.878612,3.010693
2020-05-05 00:00:00+00:00,Clear,clear-day,0,0,78.98,78.98,2.73,0.05,1013.8,9.71,...,643.5,542.5,738.5,117.0,21.0,63.0,90.0,8.0,0.882679,0.000000
2020-05-05 01:00:00+00:00,Clear,clear-day,0,0,75.87,75.87,3.20,0.06,1014.0,8.97,...,416.0,327.5,512.5,91.0,20.5,75.0,81.0,12.0,0.882679,0.130900
2020-05-05 02:00:00+00:00,Clear,clear-day,0,0,71.17,71.17,5.00,0.07,1014.1,8.08,...,94.0,75.5,116.5,20.5,18.5,86.5,72.5,5.0,0.882679,0.261799
2020-05-05 03:00:00+00:00,Clear,clear-night,0,0,65.80,65.80,5.74,0.09,1015.3,6.71,...,0.0,0.0,0.0,0.0,17.0,97.5,63.5,0.0,0.882679,0.392699
2020-05-05 04:00:00+00:00,Clear,clear-night,0,0,61.85,61.85,7.19,0.11,1016.2,7.20,...,0.0,0.0,0.0,0.0,15.5,107.5,53.0,0.0,0.882679,0.523599
2020-05-05 05:00:00+00:00,Clear,clear-night,0,0,59.41,59.41,7.66,0.13,1016.9,8.29,...,0.0,0.0,0.0,0.0,14.5,116.0,40.5,0.0,0.882679,0.654498


In [ ]:
features = ['day',
 'hour',
 'precipIntensity',
 'precipProbability',
 'precipAccumulation',
 'precipType_',
 'precipType_rain',
 'precipType_snow',
 'precipType_sleet',
 'temperature',
 'apparentTemperature',
 'dewPoint',
 'humidity',
 'pressure',
 'windSpeed',
 'windBearing',
 'windGust',
 'cloudCover',
 'uvIndex',
 'visibility',
 'ozone',
 'GHI',
 'DHI',
 'DNI']